In [ ]:
# import 필요한 라이브러리
import os
from natsort import natsorted
from pathlib import Path
import shutil
from os import listdir
from os.path import isdir
from os.path import exists
from os.path import isfile, join
import numpy as np
from tqdm.notebook import tqdm
import re
import pandas as pd
import datetime
import random
from natsort import natsorted
import secrets

In [11]:
# 필요한 함수 정의
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)


class Generator:
    def __init__(self, start_date, end_date, start_id, end_id):
        self.start_date = datetime.date(
            start_date["year"], start_date["month"], start_date["day"]
        )
        self.end_date = datetime.date(
            end_date["year"], end_date["month"], end_date["day"]
        )
        self.start_id = start_id
        self.end_id = end_id

    def dategenerate(self, qty):
        start_date = self.start_date
        end_date = self.end_date

        set_date = set()
        while len(set_date) < qty:
            time_between_dates = end_date - start_date
            days_between_dates = time_between_dates.days
            random_number_of_days = random.randrange(days_between_dates)
            random_date = start_date + datetime.timedelta(days=random_number_of_days)
            gen_date = "".join(str(random_date).split("-"))
            set_date.add(gen_date)
        return natsorted(list(set_date))

    def idgenerate(self, isconstlength=False, maxlength=3):
        set_id = set()
        for i in range(self.start_id, self.end_id):
            str_id = str(i)
            if isconstlength:
                str_id = "0" * (maxlength - len(str_id)) + str_id
            gen_id = "W" + str_id
            set_id.add(gen_id)

        return natsorted(list(set_id))


def makeColumns():
    sideType = [
        "Left",
        "Right",
    ]
    sensorType = ["ACC", "Gyro", "Mag"]
    AxisType = ["X", "Y", "Z"]
    lside = len(sideType)
    lsen = len(sensorType)
    lAxis = len(AxisType)
    col_acc = [
        f"{sideType[i//(lsen*lAxis)]}_{sensorType[(i//(lsen))%lsen]}_{AxisType[i%lAxis]}"
        for i in range(0, lsen * lAxis * lside)
    ]
    return col_acc


def genQST():
    list_Test = ["Walking", "Running", "Sitting", "Sleeping"]
    list_Location = ["Boston", "NYC", "Busan", "Seoul"]
    list_Comments = ["", "Need_discussion", "Serious_Problem"]

    data = []
    for test in list_Test:
        for _ in range(0,5):
            data.append([test,secrets.choice(list_Location),random.random(),secrets.choice(list_Comments)])
    return data

In [12]:
# 실험기간
start_date = {"year": 2022, "month": 3, "day": 1}
end_date = {"year": 2022, "month": 6, "day": 9}
# 환자 ID
start_id = 1
end_id = 21
date_exp = Generator(start_date, end_date, start_id, end_id)


In [14]:
list_patient = date_exp.idgenerate(True)
targetFolder = "Data"
data_length = 100
ensure_dir(targetFolder)
for patient in list_patient:
    ensure_dir(join(targetFolder, patient))
    # 방문일 뽑기
    visitCount = random.randrange(6)
    list_visitDate = date_exp.dategenerate(visitCount)
    # 방문 안하고 도망갔을 경우, 실험 폴더가 없음
    if len(list_visitDate) == 0:
        continue
    # 방문날짜가 정해졌을 경우, 이를 폴더로 만들기
    for visitDate in list_visitDate:
        # QST 저장
        saveDir = join(targetFolder, patient, visitDate, "QST")
        ensure_dir(saveDir)
        data_qst = genQST()
        result_qst = pd.DataFrame(data_qst,columns=["Test", "Location", "Value", "Comments"])
        result_qst.to_excel(join(saveDir, f"{patient}_{visitDate}_QST.xlsx"))
        # IMU 저장
        saveDir = join(targetFolder, patient, visitDate, "IMU")
        ensure_dir(saveDir)
        list_cols = makeColumns()
        data_subject = np.random.rand(data_length, len(list_cols))
        result_qst = pd.DataFrame(data_subject,columns=list_cols)
        result_qst.to_excel(join(saveDir, f"{patient}_{visitDate}_IMU.xlsx"))

In [17]:
saveDir = ensure_dir(join(targetFolder, patient, visitDate, "QST"))
saveDir


In [46]:
for i in range(0, 10):
    if i == 5:
        continue
    print(f"나는 {i}다")


나는 0다
나는 1다
나는 2다
나는 3다
나는 4다
나는 6다
나는 7다
나는 8다
나는 9다


In [68]:
saveDir

'Data\\W019\\20220425\\QST'

In [34]:
motherfolder = "mom"
sonfolder = "son"
grandfolder = "grandson"
join(motherfolder, sonfolder, grandfolder)


'mom\\son\\grandson'

In [85]:
# 방문 일자 뽑기
topDir = "Data"
list_patient = natsorted(
    [thing for thing in os.listdir(topDir) if isdir(join(topDir, thing))]
)
Dict_patientDate = {}
for patient in list_patient:
    list_visitDate = natsorted(
        [
            th
            for th in os.listdir(join(topDir, patient))
            if isdir(join(topDir, patient, th))
        ]
    )
    Dict_patientDate[patient] = list_visitDate


In [92]:
Dict_patientDate

{'W001': ['20220329', '20220516'],
 'W002': ['20220418', '20220511', '20220606'],
 'W003': ['20220301', '20220426', '20220504'],
 'W004': ['20220302', '20220512', '20220605'],
 'W005': ['20220312', '20220409', '20220507'],
 'W006': [],
 'W007': ['20220322'],
 'W008': ['20220410', '20220519', '20220523', '20220606', '20220608'],
 'W009': ['20220405', '20220416'],
 'W010': ['20220428'],
 'W011': [],
 'W012': ['20220316', '20220525'],
 'W013': ['20220420', '20220508', '20220509', '20220529', '20220531'],
 'W014': ['20220409'],
 'W015': ['20220401', '20220509', '20220511'],
 'W016': ['20220310'],
 'W017': ['20220410'],
 'W018': ['20220425', '20220520'],
 'W019': ['20220408', '20220425']}

In [105]:
countVisit = []
for patientDate in Dict_patientDate:
    count = len(Dict_patientDate[patientDate])
    # print(count)
    countVisit.append(count)

In [107]:
np_cv = np.array(countVisit)

In [132]:
np_cv >= 2


array([ True,  True,  True,  True,  True, False, False,  True,  True,
       False, False,  True,  True, False,  True, False, False,  True,
        True])

In [133]:
A = np.where(np_cv >= 2)
A


(array([ 0,  1,  2,  3,  4,  7,  8, 11, 12, 14, 17, 18], dtype=int64),)

In [131]:
for element in A[0]:
    print(f"iD는{element+1}")
print(f"총 인원수 {len(A[0])}")


iD는1
iD는2
iD는3
iD는4
iD는5
iD는8
iD는9
iD는12
iD는13
iD는15
iD는18
iD는19
총 인원수 12


In [113]:
np_cv[np_cv == 0]


array([0, 0])

In [137]:
A = [1, 2, 3, 4, 5]
A[0:3], A[3:]


([2, 3], [4, 5])

In [135]:
[i for i in range(0, 5)]


[0, 1, 2, 3, 4]

In [147]:
reexplist = []
for patientDate in Dict_patientDate:
    visitdata = Dict_patientDate[patientDate]
    if len(visitdata) == 0:
        continue
    for data in visitdata:
        print(int(data[4:6]) > 5)
        if int(data[4:6]) > 5:
            print(patientDate)
            reexplist.append(patientDate)


False
False
False
False
True
W002
False
False
False
False
False
True
W004
False
False
False
False
False
False
False
True
W008
True
W008
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [148]:
reexplist

['W002', 'W004', 'W008', 'W008']

In [170]:
start_date = {"year": 1, "month": 3, "day": 1}
end_date = {"year": 1, "month": 6, "day": 9}
start_date = datetime.date(
    start_date["year"], start_date["month"], start_date["day"])
end_date = datetime.date(end_date["year"], end_date["month"], end_date["day"])
set_date = set()
time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
random_date = start_date + datetime.timedelta(days=random_number_of_days)
gen_date = "".join(str(random_date).split("-"))
gen_date


'00010603'

In [ ]:
while len(set_date) < qty:
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    gen_date = "".join(str(random_date).split("-"))
    set_date.add(gen_date)
return natsorted(list(set_date))


In [151]:
a = []
wantedNumber = 10
while len(a) < wantedNumber:
    inputValue = 10
    a.append(inputValue)


In [150]:
a

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [153]:
A = set()
A.add(10)
A.add(10)
A.add(10)
A.add(10)
A.add(10)
A.add(10)
A

{10}

In [154]:
A = []
A.append(10)
A.append(10)
A.append(10)
A.append(10)
A.append(10)
A.append(10)
A.append(10)
A

[10, 10, 10, 10, 10, 10, 10]

In [ ]:
num_file = 10
data_length = 101
num_feature = 36
save_dir = "./IMU_EXP"
ensure_dir(save_dir)
for i in range(0, num_file):
    # 피험자 이름 생성
    name_subject = names.get_full_name()
    # 오늘 날짜
    str_date = date.today().strftime("%B_%d_%Y")
    # 데이터 생성
    data_subject = np.random.rand(data_length, num_feature)
    # columns 생성
    list_col = makeColumns()
    # columns 별 데이터 생성 (random 생성, 0에서 1사이 데이터)
    df_subject = pd.DataFrame(data_subject, columns=list_col)
    # 각파일별 이름 저장
    df_subject.to_excel(
        os.path.join(save_dir, f"{str_date}_{name_subject}_IMUData.xlsx")
    )

In [ ]:
def makeColumns():
    sideType = ["Front_Left", "Front_Right", "Back_Left", "Back_Right"]
    sensorType = ["ACC", "Gyro", "Mag"]
    AxisType = ["X", "Y", "Z"]
    lside = len(sideType)
    lsen = len(sensorType)
    lAxis = len(AxisType)
    col_acc = [
        f"{sideType[i//(lsen*lAxis)]}_{sensorType[(i//(lsen))%lsen]}_{AxisType[i%lAxis]}"
        for i in range(0, lsen * lAxis * lside)
    ]
    return col_acc

In [ ]:
result_qst = pd.DataFrame(columns=["Test", "Location", "Value", "Comments"])


In [4]:
import secrets
def genQST():
    list_Test = ["Walking", "Running", "Sitting", "Sleeping"]
    list_Location = ["Boston", "NYC", "Busan", "Seoul"]
    list_Comments = ["", "Need_discussion", "Serious_Problem"]

    data = []
    for test in list_Test:
        for _ in range(0,5):
            data.append([test,secrets.choice(list_Location),random.random(),secrets.choice(list_Comments)])
    return data
            

In [3]:
random.random()

0.7979270455435613

In [5]:
genQST()

[['Walking', 'Boston', 0.7710466817002023, 'Serious_Problem'],
 ['Walking', 'Seoul', 0.8260328778525939, ''],
 ['Walking', 'NYC', 0.2146740295788303, 'Need_discussion'],
 ['Walking', 'NYC', 0.7078264435541809, ''],
 ['Walking', 'Seoul', 0.19438441478676216, 'Serious_Problem'],
 ['Running', 'Busan', 0.8142088739062248, 'Need_discussion'],
 ['Running', 'NYC', 0.8247569320808998, 'Serious_Problem'],
 ['Running', 'Seoul', 0.9322240726721073, ''],
 ['Running', 'Boston', 0.4440395322450136, 'Serious_Problem'],
 ['Running', 'Seoul', 0.4620455126079459, 'Need_discussion'],
 ['Sitting', 'Boston', 0.6245605422767767, ''],
 ['Sitting', 'Boston', 0.9691327087665509, 'Serious_Problem'],
 ['Sitting', 'Busan', 0.7625686145223394, 'Serious_Problem'],
 ['Sitting', 'NYC', 0.7160066024890205, ''],
 ['Sitting', 'NYC', 0.1988867851101208, 'Need_discussion'],
 ['Sleeping', 'NYC', 0.5555160492747617, 'Serious_Problem'],
 ['Sleeping', 'Busan', 0.08078836383776122, 'Serious_Problem'],
 ['Sleeping', 'Boston', 0